**2. COLLECT THE COMPLEXES DATA FROM THE API ENDPOINTS**

In [7]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=2gS1R2oRxMyHyBmFK4cJRK7N3HZIJRMXfDiVYRdv"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-05-02T18:36:46+00:00","complexes":[{"id":"sr:complex:705","name":"Nacional","venues":[{"id":"sr:venue:70045","name":"Cancha Central","city_name":"Santiago","country_name":"Chile","country_code":"CHL","timezone":"America\/Santiago"}]},{"id":"sr:complex:1078","name":"Estadio de la Cartuja","venues":[{"id":"sr:venue:74856","name":"Centre Court","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"},{"id":"sr:venue:74858","name":"Court One","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"}]},{"id":"sr:complex:1495","name":"Sibur Arena","venues":[{"id":"sr:venue:1496","name":"COURT 1","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:1500","name":"CENTER COURT","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:62149","name":"Sibur Arena","city_na

In [ ]:
import json
data = json.loads(response.text)
complexes = data['complexes']

In [10]:
complexes_data = []
venues_data = []

for complex in complexes:
    complexes_data.append({
        "complex_id": complex['id'],
        "complex_name": complex['name']
    })

    for venue in complex.get('venues', []):
        venues_data.append({
            "venue_id": venue['id'],
            "venue_name": venue['name'],
            "city_name": venue['city_name'],
            "country_name": venue['country_name'],
            "country_code": venue['country_code'],
            "timezone": venue['timezone'],
            "complex_id": complex['id']
        })

In [ ]:
import pandas as pd
complexes_df = pd.DataFrame(complexes_data)
venues_df = pd.DataFrame(venues_data)

In [12]:
import mysql.connector

conn_mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Joker@212814"
)
cursor_mysql = conn_mysql.cursor()
print("MySQL connection established!")

MySQL connection established!


In [13]:
cursor_mysql.execute("CREATE DATABASE IF NOT EXISTS tennis_db;")
print("MySQL database 'tennis_db' created successfully!")

MySQL database 'tennis_db' created successfully!


In [ ]:
cursor_mysql.execute("USE tennis_db;")

cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS complexes (
        complex_id VARCHAR(50) PRIMARY KEY,
        complex_name VARCHAR(100) NOT NULL
    );
""")


cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS venues (
            venue_id VARCHAR(50) PRIMARY KEY,
            venue_name VARCHAR(100) NOT NULL,
            city_name VARCHAR(100) NOT NULL,
            country_name VARCHAR(100) NOT NULL,
            country_code CHAR(3) NOT NULL,
            timezone VARCHAR(100) NOT NULL,
            complex_id VARCHAR(50),
            FOREIGN KEY (complex_id) REFERENCES complexes(complex_id)
                );
""")
conn_mysql.commit()
print("Tables 'complexes' and 'venues' created successfully in MySQL!")


Tables 'complexes' and 'venues' created successfully in MySQL!


In [23]:
for index, row in complexes_df.iterrows():
    cursor_mysql.execute("""
        INSERT INTO complexes (complex_id, complex_name) 
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE 
                            complex_name = VALUES(complex_name)      
    """, (row['complex_id'], row['complex_name'])
    )

for index, row in venues_df.iterrows():
    cursor_mysql.execute("""
        INSERT INTO venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
                        venue_name = VALUES(venue_name), 
                        city_name = VALUES(city_name), 
                        country_name = VALUES(country_name), 
                        country_code = VALUES(country_code), 
                        timezone = VALUES(timezone),
                        complex_id = VALUES(complex_id)
    """, (row['venue_id'], row['venue_name'], row['city_name'], row['country_name'], row['country_code'], row['timezone'], row['complex_id'])
    )

conn_mysql.commit()
print("Data inserted successfully!")

Data inserted successfully!


1.List all venues along with their associated complex name

In [16]:
from tabulate import tabulate

query1 = """
SELECT v.venue_name, c.complex_name
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id;
"""
cursor_mysql.execute(query1)
result = cursor_mysql.fetchall()

headers = ["Venue Name", "Complex Name"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+------------------------------------------------------------+------------------------------------------------------------+
| Venue Name                                                 | Complex Name                                               |
+============================================================+============================================================+
| Estadio Monumental                                         | Club Tennis Las Terrazas de Miraflores                     |
+------------------------------------------------------------+------------------------------------------------------------+
| Stadium                                                    | Shenzhen International Tennis Center                       |
+------------------------------------------------------------+------------------------------------------------------------+
| Court 2                                                    | Shenzhen Longgang Sports Center                            |
+-------

2.Count the number of venues in each complex

In [17]:
from tabulate import tabulate

query2 = """
SELECT c.complex_name , COUNT(v.venue_id) AS venue_count
FROM complexes c
LEFT JOIN venues v ON v.complex_id = c.complex_id
GROUP BY c.complex_name;
"""
cursor_mysql.execute(query2)
result = cursor_mysql.fetchall()

headers = ["Complex Name", "Venue count"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+------------------------------------------------------------+---------------+
| Complex Name                                               |   Venue count |
+============================================================+===============+
| Kindarena                                                  |             4 |
+------------------------------------------------------------+---------------+
| Estadio de la Cartuja                                      |             2 |
+------------------------------------------------------------+---------------+
| Estadio German Becker                                      |             4 |
+------------------------------------------------------------+---------------+
| Palais des sports de Gerland                               |             7 |
+------------------------------------------------------------+---------------+
| Sibur Arena                                                |             4 |
+---------------------------------------------------

3.Get details of venues in a specific country (e.g., Chile)

In [18]:
from tabulate import tabulate

query3 = """
SELECT venue_id ,venue_name,city_name
FROM venues
WHERE country_name = 'Chile';
"""
cursor_mysql.execute(query3)
result = cursor_mysql.fetchall()

headers = ["Venue ID", "Venue Name", "City Name"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+----------------+-------------------------+--------------+
| Venue ID       | Venue Name              | City Name    |
+================+=========================+==============+
| sr:venue:13830 | Cancha 1                | Santiago     |
+----------------+-------------------------+--------------+
| sr:venue:13832 | Cancha 3                | Santiago     |
+----------------+-------------------------+--------------+
| sr:venue:15858 | Centre Court            | Santiago     |
+----------------+-------------------------+--------------+
| sr:venue:15860 | Court 16                | Santiago     |
+----------------+-------------------------+--------------+
| sr:venue:15862 | Court 15                | Santiago     |
+----------------+-------------------------+--------------+
| sr:venue:15864 | Court 11                | Santiago     |
+----------------+-------------------------+--------------+
| sr:venue:1602  | Cancha Central          | Santiago     |
+----------------+----------------------

4.Identify all venues and their timezones

In [19]:
from tabulate import tabulate

query4 = """
SELECT venue_id ,venue_name,city_name,timezone
FROM venues;
"""
cursor_mysql.execute(query4)
result = cursor_mysql.fetchall()

headers = ["Venue ID", "Venue Name", "City Name", "Timezone"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+----------------+------------------------------------------------------------+----------------------------+--------------------------------+
| Venue ID       | Venue Name                                                 | City Name                  | Timezone                       |
+================+============================================================+============================+================================+
| sr:venue:1008  | Estadio Monumental                                         | Lima                       | America/Lima                   |
+----------------+------------------------------------------------------------+----------------------------+--------------------------------+
| sr:venue:10390 | Stadium                                                    | Shenzhen                   | Asia/Shanghai                  |
+----------------+------------------------------------------------------------+----------------------------+--------------------------------+
| sr:v

5.Find complexes that have more than one venue

In [20]:
from tabulate import tabulate

query5 = """
SELECT c.complex_name , COUNT(v.venue_id) AS venue_count
from complexes c
LEFT JOIN venues v ON c.complex_id = v.complex_id
GROUP BY c.complex_name
having COUNT(v.venue_id) > 1;  
"""
cursor_mysql.execute(query5)
result = cursor_mysql.fetchall()

headers = ["Complex Name", "Venue Count"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+------------------------------------------------------------+---------------+
| Complex Name                                               |   Venue Count |
+============================================================+===============+
| Kindarena                                                  |             4 |
+------------------------------------------------------------+---------------+
| Estadio de la Cartuja                                      |             2 |
+------------------------------------------------------------+---------------+
| Estadio German Becker                                      |             4 |
+------------------------------------------------------------+---------------+
| Palais des sports de Gerland                               |             7 |
+------------------------------------------------------------+---------------+
| Sibur Arena                                                |             4 |
+---------------------------------------------------

6.List venues grouped by country

In [21]:
from tabulate import tabulate

query6 = """
SELECT country_name , venue_name
FROM venues
order BY country_name;  
"""
cursor_mysql.execute(query6)
result = cursor_mysql.fetchall()

headers = ["Country Name", "Venue Name"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+----------------------+------------------------------------------------------------+
| Country Name         | Venue Name                                                 |
+======================+============================================================+
| Argentina            | Cancha 4                                                   |
+----------------------+------------------------------------------------------------+
| Argentina            | Court 12                                                   |
+----------------------+------------------------------------------------------------+
| Argentina            | Club Nautico Hacoaj - Center Court                         |
+----------------------+------------------------------------------------------------+
| Argentina            | Cancha 2                                                   |
+----------------------+------------------------------------------------------------+
| Argentina            | Cancha Central               

7.Find all venues for a specific complex (e.g., Nacional)

In [22]:
from tabulate import tabulate

query7 = """
SELECT v.venue_name
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id
wHERE c.complex_name = 'Nacional';  
"""
cursor_mysql.execute(query7)
result = cursor_mysql.fetchall()

headers = ["Country Name", "Venue Name"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+----------------+
| Country Name   |
+================+
| Cancha Central |
+----------------+
